<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/convert2png.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **BATCH CONVERT FROM DNG TO PNG**

INITIALIZATION

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import os
import os.path
import imageio
import numpy as np
!pip install rawpy
import rawpy

DEFINE DIRECTORIES

In [14]:
dataset_name = "0degree_LED"
your_dir_path = '/content/gdrive/MyDrive/DATA/0_degree_LED' # String. The path of your directory containing all your subdirectories
new_dir_path = f'/content/{dataset_name}' # The path of your new directory with the same architecture as the previous one, but with cropped and aligned faces

MAIN CODE

In [ ]:
for subfolder in next(os.walk(your_dir_path))[1] : # Gives the list of all subdirectories inside the parent directory
  os.makedirs(os.path.join(new_dir_path, subfolder)) # Creates the new subdirectory. Note that it will also create new_dir_path, so there's no need to add a line os.makedirs(new_dir_path)
  for file in os.listdir(os.path.join(your_dir_path, subfolder)) : # Gives the list of all files inside the 'subfolder' directory
    if file.endswith('.dng'):  
      print("converting " + str(file))
      size = len(file)
      filename = file[:size - 4]

      raw = rawpy.imread(os.path.join(your_dir_path, subfolder, file))
      # Postprocessing, i.e demosaicing here, will always 
      #change the original pixel values. Typically what you want
      # is to get a linearly postprocessed image so that roughly 
      #the number of photons are in linear relation to the pixel values. 
      #You can do that with:
      rgb = raw.postprocess(gamma=(1,1), no_auto_bright=True, output_bps=8)

      #Apply gamma corrections: gamma values greater than 1 will shift the image histogram towards left and the output image will be darker than the input image. On the other hand, for gamma values less than 1, the histogram will shift towards right and the output image will be brighter than the input image.
      gamma_corrected_rgb = exposure.adjust_gamma(rgb, gamma=0.5, gain=1)
      image=gamma_corrected_rgb

      imageio.imsave(os.path.join(new_dir_path, subfolder, f'{filename}.png'), image)


COPY TO DRIVE

In [17]:
save_path = f"/content/gdrive/MyDrive/DATA/PNG/{dataset_name}"
if not os.path.exists(save_path):
  os.makedirs(save_path)

oripath = new_dir_path + "/."
!cp -a "{oripath}" "{save_path}" # copies files to google drive

DELETE TEST FOLDER

In [18]:
from shutil import rmtree # deletes a folder
rmtree(new_dir_path)